# Imports


In [1]:
!pip uninstall -y numpy
!pip install numpy==1.23.5


  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 137, in get_requires_for_build_

In [ ]:
import numpy
import tensorflow


  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)


In [3]:
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

/Users/nicolehao/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


AttributeError: module 'numpy' has no attribute 'typeDict'

# Loading

In [ ]:
# Load the second uploaded notebook

notebook_path_2 = "./Parallel CNN-Based Transient Stability Analysis.ipynb"
with open(notebook_path_2, 'r', encoding='utf-8') as f:
    notebook_2 = nbformat.read(f, as_version=4)

# Extract code cells from the second notebook
code_cells_2 = [cell['source'] for cell in notebook_2.cells if cell.cell_type == 'code']
code_cells_2[:3]  # Show first few code cells to understand the structure



# Scaling Test

In [ ]:
# Scaling Test (Data Size Only)

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Split data once to reuse same test set
X_train_full, X_test, y_train_full, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

fractions = [0.1, 0.25, 0.5, 0.75, 1.0]
results = []

for frac in fractions:
    size = int(len(X_train_full) * frac)
    X_train = X_train_full[:size]
    y_train = y_train_full[:size]

    model = build_model(X_train.shape[1:])

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    duration = time.time() - start_time

    y_pred = model.predict(X_test).flatten() > 0.5
    acc = accuracy_score(y_test, y_pred)

    results.append((frac, size, duration, acc))
    print(f"Data Fraction: {frac}, Train Size: {size}, Time: {duration:.2f}s, Accuracy: {acc:.4f}")




In [ ]:
# Display results as DataFrame
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])
print(df_results)


In [ ]:
# Plot
df_results.plot(x="Train Size", y=["Time (s)", "Accuracy"], marker='o', title="Scaling Test Results")
plt.grid(True)
plt.show()
